In [66]:
import numpy as np

from qibo.symbols import X,Y,Z
from qibo.hamiltonians import SymbolicHamiltonian
from qibo import set_backend
from qibo import models

from itertools import product
import random

random.seed(13)

# Generating the dataset

In [67]:
# Define the number of items
n_items = 6

# Define ranges
duration_range = [1, 7]
values_range = [5, 15]
max_duration_percentage = 0.7

# Fill the weights and values 
duration = [random.randint(duration_range[0], duration_range[1]) for _ in range(n_items)]
values  = [random.randint(values_range[0], values_range[1]) for _ in range(n_items)]

# Compute the maximum allowed weight
max_duration = int(max_duration_percentage * sum(duration))


# Print the instance
print("-" * 20)
print("Instance Details:")
print("-" * 20)
print(f"Duration                 : {duration}")
print(f"Values                   : {values}")
print(f"Total duration           : {sum(duration)}")
print(f"Maximum allowed duration : {max_duration}")

--------------------
Instance Details:
--------------------
Duration                 : [3, 3, 6, 6, 7, 7]
Values                   : [7, 15, 8, 15, 7, 8]
Total duration           : 32
Maximum allowed duration : 22


In [68]:
def build_cost_hamiltonian(values: list, duration: list, max_duration: int) -> SymbolicHamiltonian:
    """
    This function should be filled to build the problem cost hamiltonian.

    Args:
        values (list[int]): the list of values.
        duration (list[int]): the list of durations. 
        max_duration (int): the maximum value of the allowed duration.
    """
    nQubits = len(values)
    cost_hamiltonian = (-1) * sum([values[i] * (1- Z(i))/2 for i in range(nQubits)])

    return SymbolicHamiltonian(cost_hamiltonian)

In [69]:
# number of qubits
nQubits = len(values)

In [70]:
def value_from_eigvec(eigvec:str) -> float: return sum([values[int(i)] for i in eigvec if eigvec[int(i)] == '1'])
def duration_from_eigvec(eigvec:str) -> float: return sum([duration[int(i)] for i in eigvec if eigvec[int(i)] == '1'])

### Diagonalize the cost hamiltonian and examen its eigenvalues and eigenvectors. 
- Show that the ground state eigenvector does actually correspond to the ground state energy of the problem. (Note: the ground state could be degenerate)

In [71]:
# set the backend used for the calculation 
set_backend("numpy", platform=None)


# create the cost Hamiltonian for the given graph
cost_hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)

ham_matrix = cost_hamiltonian.matrix

eig_val, eig_vec = np.linalg.eig(ham_matrix)
eig_vec = ["{0:0{bits}b}".format(i.argmax(), bits=nQubits) for i in eig_vec]

vec = zip(eig_val, eig_vec)
diagonalized_solution = sorted(vec, key=lambda x: x[0])

print()
# print(diagonalized_solution)

for energy, eigvec in diagonalized_solution:
    print("energy = {:7.3f} | value = {:2} | duration = {:2}".format(np.real(energy), value_from_eigvec(eigvec), duration_from_eigvec(eigvec)))

[Qibo 0.2.2|INFO|2024-05-04 16:35:40]: Using numpy backend on /CPU:0
[Qibo 0.2.2|WARNING|2024-05-04 16:35:40]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.



energy = -60.000 | value = 90 | duration = 18
energy = -53.000 | value = 75 | duration = 15
energy = -53.000 | value = 82 | duration = 18
energy = -52.000 | value = 82 | duration = 18
energy = -52.000 | value = 82 | duration = 18
energy = -46.000 | value = 60 | duration = 12
energy = -45.000 | value = 60 | duration = 12
energy = -45.000 | value = 60 | duration = 12
energy = -45.000 | value =  7 | duration =  3
energy = -45.000 | value = 74 | duration = 18
energy = -45.000 | value = 82 | duration = 18
energy = -45.000 | value = 74 | duration = 18
energy = -44.000 | value = 74 | duration = 18
energy = -38.000 | value =  0 | duration =  0
energy = -38.000 | value = 45 | duration =  9
energy = -38.000 | value = 60 | duration = 12
energy = -38.000 | value = 45 | duration =  9
energy = -38.000 | value = 14 | duration =  6
energy = -38.000 | value = 74 | duration = 18
energy = -37.000 | value = 45 | duration =  9
energy = -37.000 | value = 14 | duration =  6
energy = -37.000 | value = 14 | d

### Solve this hamiltonian using the QAOA algorithm.

        step 1. Define the Mixing hamiltonian.

In [72]:
def indicator_operator(state:list):
    """Returns a projector on the subspace spanned by the quantum state `state`."""
    operator = 1
    for b in state:
        operator *= ((1 - Z(b))/2 + np.mod(b+1,2))

    return operator

def switching_operator(in_state:np.ndarray,out_state:np.ndarray):
    """Returns an operator that maps computational basis state `in_state` to `out_state`."""
    operator = 1
    for iQubit,b in enumerate(in_state - out_state):
        if b != 0:
            operator *= X(iQubit)

    return operator

def build_mixer_hamiltonian(nQubits:int,duration:list,subspace:list) -> SymbolicHamiltonian:
    '''
    build the mixer hamiltonian for the given graph.

    args:
        nQubits: The number of qubits the hamiltonian acts upon.
        duration: List containing the duration associated with each task.
        subspace: A list of the vectors in the subspace that fulfills the constraint.

    returns:
        The mixer hamiltonian of the given graph

    '''
    # sanity check
    if not nQubits == len(duration): raise ValueError("Every qubit must have a duration that is associated with the corresponding task!")
    # dimension of the subspace
    N = len(subspace)

    mixing_hamiltonian = sum([indicator_operator(subspace[iState]) * switching_operator(subspace[iState],subspace[(iState+1) % N]) for iState in range(N)])

    return SymbolicHamiltonian(mixing_hamiltonian)

        Step 2. Prepare the initial state that satisfies the inequality constraint

In [73]:
hilbert_space = np.array([[int(b) for b in np.binary_repr(k,width=nQubits)] for k in range(2**nQubits)])

subspace_mask = np.array([sum(hilbert_space[iState,:] * duration) <= max_duration for iState in range(hilbert_space.shape[0])])
subspace = hilbert_space[subspace_mask,:]

initial_state = subspace_mask.astype(float) / np.sqrt(sum(subspace_mask))

        Step 3. Run the QAOA algorithm.

In [74]:
hamiltonian = build_cost_hamiltonian(values=values, duration=duration, max_duration=max_duration)
mixer_hamiltonian = build_mixer_hamiltonian(nQubits,duration,subspace)

# create QAOA model given the Hamiltonians
qaoa = models.QAOA(hamiltonian=hamiltonian, mixer=mixer_hamiltonian)

# optimize using random initial variational parameters with four layers 
n_layers = 5
# initial_parameters = 0.01 * np.random.random(n_layers * 2)
initial_parameters =  0.01 *  (2 * np.random.random(n_layers * 2) - 1) * np.pi
print(initial_parameters)

# run the QAOA optimization with the initial parameters and the hamiltonians defined 

# Define random initial variational parameters with four layers 
n_layers = 4
initial_parameters = 0.01 * np.random.random(n_layers * 2)


"""
supported optimization Method: 
- Nelder-Mead 
- parallel_L-BFGS-B
- Powell
- CG
- cma
- sgd
- L-BFGS-B
- Newton-CG
- COBYLA
- BFGS
- trust-constr
"""
method = "Powell"


best_energy, final_parameters, _ = qaoa.minimize(
    initial_parameters,
    method=method,
    initial_state=initial_state
)

print("best energy: ", best_energy)

[-0.01685163 -0.00635232 -0.0082818   0.02327075  0.02327841 -0.01527427
 -0.003817   -0.00135149  0.02748408 -0.03117137]
best energy:  -135.74246149439523
